In [1]:
#https://pair-code.github.io/lit/
#https://www.tensorflow.org/tensorboard/get_started
# https://www.tensorflow.org/tensorboard/dataframe_api
# https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
#https://keras.io/guides/keras_tuner/visualize_tuning/

#https://github.com/technqvi/TimeSeriesML-FinMarket/blob/main/MultiVarToManyOutputLSTM.ipynb



In [2]:
# import necessary libraries

#https://codelabs.developers.google.com/codelabs/fraud-detection-ai-explanations?hl=en#0
#he Explainable AI SDK and Copy Model to Deploy

#https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/introduction_to_tensorflow/solutions/preprocessing_layers.ipynb
#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom-tabular-bq-managed-dataset.ipynb
#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_local_trained_models.ipynb

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,DenseFeatures

from datetime import date, timedelta, datetime # Date Functions
import time


from google.cloud import bigquery
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# from google.cloud import aiplatform as vertex_ai
from tensorflow.python.keras.utils import data_utils

import tensorboard as tb

# from tensorboard.plugins.hparams import api as hp

print(tf.__version__)
print(tb.__version__)

2.12.0
2.12.0


In [4]:
# major_ver, minor_ver, _ = version.parse(tb.__version__).release
# assert major_ver >= 2 and minor_ver >= 3, \
#     "This notebook requires TensorBoard 2.3 or later."
# print("TensorBoard version: ", tb.__version__)

In [5]:
# Clear any logs from previous runs
# !rm -rf ./logs/ 

In [6]:
cateCols=['sla','product_type','brand','service_type','incident_type']
numbericCols=['open_to_close_hour','response_to_resolved_hour']
unusedCols=['severity_id','severity_name','label_binary_severity']

# cateCols=['service_type','product_type','incident_type',]
# numbericCols=[]
# unusedCols=['severity_id','severity_name','label_binary_severity'. \
#            'open_to_close_hour','response_to_resolved_hour' .\
#             'sla'
#            ]

# cateCols=['sla','product_type','service_type','incident_type']
# numbericCols=['open_to_close_hour']
# unusedCols=['severity_id','severity_name','label_binary_severity','brand','response_to_resolved_hour']

labelCol='label_multi_severity'

main_metric='accuracy'
main_objective=f'val_{main_metric}'


objective_to_tued_and_monitor='val_loss' 
# objective_to_tued_and_monitor=main_objective

seed=1932

EPOCHS =100
BATCH_SIZE = 32
unitList=[16,32,64,128]
dropOutList= [0.1, 0.2]
lrList=[0.01,0.001,0.0001]
nAtleastMaxTrials=10
nExecutions_per_trial=3  # 3,5
nEarlyPatience=11


# EPOCHS =20
# BATCH_SIZE = 32
# unitList=[32]
# dropOutList= [0.1,0.2]
# lrList=[0.01,0.001]
# nAtleastMaxTrials=10
# nExecutions_per_trial=1  # 3,5
# nEarlyPatience=10

model_tuned_dir='tuned_model_V2'

# df['label_multi_severity'] =df['severity_name'].map({'Cosmatic':0,'Minor': 1, "Major": 2, "Critical": 3}) 

# Load Data

In [7]:
projectId='pongthorn'
client = bigquery.Client(project=projectId)
dataset_id='SMartML'

train_name='train_incident'
validation_name='validation_incident'
test_name='test_incident'

train_table_id=f"{projectId}.{dataset_id}.{train_name}"
val_tabel_id=f"{projectId}.{dataset_id}.{validation_name}"
test_tabel_id=f"{projectId}.{dataset_id}.{test_name}"

In [8]:
# def load_ml_data(data_path):
#  df=pd.read_csv(data_path)
#  df =df.drop(columns=unusedCols)
#  return df

# root_path='../../data'    
# train = load_ml_data(f"{root_path}/train_incident.csv")
# # val=train.copy()
# val=load_ml_data(f"{root_path}/validation_incident.csv")
# # test =val.copy()
# test =load_ml_data(f"{root_path}/test_incident.csv")

def load_data_bq(sql:str):
 
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 df =df.drop(columns=unusedCols)
  
 return df
    

train=load_data_bq(f"SELECT * FROM {train_table_id}")
val=load_data_bq(f"SELECT * FROM {val_tabel_id}")
test=load_data_bq(f"SELECT * FROM {test_tabel_id}")

print(train.shape)
print(val.shape)
print(test.shape)


labelList=list(train[labelCol].unique())
print(labelList)
nLabel=len(labelList)
print(f"No target label : {nLabel}")

# sr_predict=df.iloc[-1,:]
# df=df.iloc[0:len(df)-1,:]
                 
print(train.info())
train.tail()

(2731, 8)
(341, 8)
(342, 8)
[2, 1, 0, 3]
No target label : 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2731 entries, 0 to 2730
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   sla                        2731 non-null   object 
 1   product_type               2731 non-null   object 
 2   brand                      2731 non-null   object 
 3   service_type               2731 non-null   object 
 4   incident_type              2731 non-null   object 
 5   open_to_close_hour         2731 non-null   float64
 6   response_to_resolved_hour  2731 non-null   float64
 7   label_multi_severity       2731 non-null   Int64  
dtypes: Int64(1), float64(2), object(5)
memory usage: 173.5+ KB
None


,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour,label_multi_severity
2726,24x7 4Hrs Response Time,Security,Trend Micro,Request,General Incident,0.333333,0.333333,0
2727,24x7 6Hrs Resolution Time,Software,Trend Micro,Request,General Incident,0.883333,0.716667,0
2728,24x7 4Hrs Response Time,Security,Trend Micro,Request,OS / Firmware,16.666667,16.666667,0
2729,24x7 4Hrs Response Time,Software,Trend Micro,Request,General Incident,7.400000,7.283333,0
2730,24x7 6Hrs Resolution Time,Software,Trend Micro,Request,General Incident,5.483333,5.483333,0


In [9]:
def CalPctEachTargetClass(dfx,colSev,colPctSev):
    dfClassSummary=dfx.groupby([labelCol]).size().to_frame(colSev)
    dfClassSummary[colPctSev]= dfClassSummary[colSev]/dfClassSummary[colSev].sum() *100
    dfClassSummary=dfClassSummary.round(0)
    return dfClassSummary

pctDF1=CalPctEachTargetClass(train,'Train-No-Severity','Train-%-Severity')
pctDF2=CalPctEachTargetClass(val,'Val-No-Severity','Val-%-Severity')
pdcDF3=CalPctEachTargetClass(test,'Test-No-Severity','Test-%-Severity')
pctDF=pd.concat([pctDF1,pctDF2,pdcDF3],axis=1)

pctDF

,Train-No-Severity,Train-%-Severity,Val-No-Severity,Val-%-Severity,Test-No-Severity,Test-%-Severity
label_multi_severity,,,,,,
0,511,19.0,53,16.0,55,16.0
1,778,28.0,97,28.0,92,27.0
2,1341,49.0,180,53.0,181,53.0
3,101,4.0,11,3.0,14,4.0


# Process Data  Layer

In [10]:
def multiple_label_df_to_dataset(dataframe, shuffle=True, batch_size=32):
  features = dataframe.copy()
  labels = features.pop(labelCol)
  labels  = tf.keras.utils.to_categorical(labels, num_classes=nLabel)
    
  ds = tf.data.Dataset.from_tensor_slices(( dict(features), labels ))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(features))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds
     
     

In [11]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for our feature.
  normalizer = preprocessing.Normalization(axis=None)

  # Prepare a Dataset that only yields our feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [12]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a StringLookup layer which will turn strings into integer indices
  if dtype == 'string':
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_tokens=max_tokens)

  # Prepare a Dataset that only yields our feature
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Create a Discretization for our integer indices.
  encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply one-hot encoding to our indices. The lambda function captures the
  # layer so we can use them, or include them in the functional model later.
  return lambda feature: encoder(index(feature))
     

In [13]:
batch_size =32
train_ds = multiple_label_df_to_dataset (train, batch_size=batch_size)
val_ds = multiple_label_df_to_dataset(val, batch_size=batch_size)
test_ds = multiple_label_df_to_dataset(test, batch_size=batch_size)
# for element in train_ds.as_numpy_iterator():
#     print(element)

In [14]:
all_inputs = []
encoded_features = []
numInputFeatToInitNodeUnit=0
# Numeric features.
for header in numbericCols:
  print(header)  
  stat_data=train[header].describe()
  print(f"header: Mean={stat_data['mean']} and Std={stat_data['std']}") 
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)
  print("========================================================================")
    
numInputFeatToInitNodeUnit=numInputFeatToInitNodeUnit+len(numbericCols)
 
# Categorical features encoded as string.
categorical_cols = cateCols
for header in categorical_cols:
    
  listCateItem=train[header].unique()
  noCateItem=len(listCateItem)
  numInputFeatToInitNodeUnit=numInputFeatToInitNodeUnit+noCateItem +1  # last 1 is unknow  
  print(f"{header} = {noCateItem} : {listCateItem}")    
    
  print(header)  
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  encoding_layer = get_category_encoding_layer(header, train_ds, dtype='string')
                                        
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)



open_to_close_hour
header: Mean=120.7417429512999 and Std=339.9325107996055
response_to_resolved_hour
header: Mean=92.4087208592701 and Std=292.40800375934094
sla = 7 : ['24x7 4Hrs Resolution Time' '24x7 4Hrs Response Time'
 '24x7 6Hrs Resolution Time' '8x5 4Hrs Response Time'
 '9x5 NBD 4Hrs Response Time' '24x7 6Hrs Response Time' 'Non MA']
sla
product_type = 13 : ['Other' 'Printer' 'Storage' 'Software' 'Server' 'Access Point' 'Switch'
 'Hardware' 'Tape Library' 'Security' 'Service' 'Notebook' 'Firewall']
product_type
brand = 28 : ['F5' 'HP' 'EMC' 'HPE' 'IBM' 'YIP' 'DELL' 'QNAP' 'Cisco' 'SAPB1' 'Veeam'
 'NetApp' 'Oracle' 'VMWare' 'Zabbix' 'eUnite' 'Nutanix' 'Red Hat'
 'Veritas' 'Alfresco' 'Broadcom' 'Fortinet' 'Commvault' 'Microsoft'
 'Palo Alto' 'CIMCO-CMMS' 'CheckPoint' 'Trend Micro']
brand
service_type = 2 : ['Incident' 'Request']
service_type
incident_type = 21 : ['General Incident' 'Network Adapter Failure' 'Software'
 'Maintenance System' 'Hard Disk Drive Failure' 'Power Supply 

# Tune HyperParameter By Keras Tuner

In [15]:
unitList.append(numInputFeatToInitNodeUnit)
unitList.append(numInputFeatToInitNodeUnit*2)
print(unitList,dropOutList,lrList)
nMax_trials=len(unitList)*len(dropOutList)*len(lrList)
if nMax_trials<nAtleastMaxTrials:
 nMax_trials=nAtleastMaxTrials 
print(nMax_trials)

[16, 32, 64, 128, 78, 156] [0.1, 0.2] [0.01, 0.001, 0.0001]
36


In [16]:
#https://github.com/technqvi/TimeSeriesML-FinMarket/blob/main/lstm-tune-dev/Tuned-MultiVarToManyOutputLSTM.ipynb
#https://www.tensorflow.org/tutorials/keras/keras_tuner
#https://keras.io/guides/keras_tuner/getting_started/
import keras_tuner as kt
from keras_tuner.engine.hyperparameters import HyperParameters
tune_folder="IncidentMLP"

In [17]:
t_Start=time.time()

print(f"Start tund at {datetime.now()}")
print("=============================================================")
buildtime = datetime.now().strftime('%d%m%y_%H%M')
project_model=f"{tune_folder}_{buildtime}"
print(project_model)

Start tund at 2023-11-11 01:42:16.446807
IncidentMLP_111123_0142


In [18]:
def build_model_for_tuning(hp):
    
    all_features = tf.keras.layers.concatenate(encoded_features)
    
    x = tf.keras.layers.Dense(hp.Choice('units', unitList), activation="relu")(all_features)
    
    x = tf.keras.layers.Dropout(hp.Choice('Dropout_rate',dropOutList))(x)
    
    output = tf.keras.layers.Dense(nLabel,activation=tf.nn.softmax)(x)
    
    model = tf.keras.Model(all_inputs, output)
    # model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=[main_metric])
    
    hp_learning_rate = hp.Choice('learning_rate', values=lrList) 
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(),metrics=[main_metric])
    return model

In [ ]:
tuner = kt.BayesianOptimization(
## tuner = kt.Hyperband(
# tuner = keras_tuner.RandomSearch(  #main_objective
    build_model_for_tuning, objective=objective_to_tued_and_monitor,seed=seed,
    max_trials=nMax_trials,executions_per_trial=nExecutions_per_trial,
    directory=f"tuning/{tune_folder}/",project_name= project_model)

stop_early = tf.keras.callbacks.EarlyStopping(monitor=objective_to_tued_and_monitor, patience=nEarlyPatience)
tuner.search(train_ds, batch_size=BATCH_SIZE,epochs=EPOCHS,validation_data=val_ds)

# log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tsb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

# stop_early = tf.keras.callbacks.EarlyStopping(monitor=objective_to_tued_and_monitor, patience=nEarlyPatience)
# tuner.search(train_ds, batch_size=BATCH_SIZE,epochs=EPOCHS,validation_data=val_ds,callbacks=[stop_early, tsb_callback])
    


Trial 3 Complete [00h 01m 19s]
val_loss: 0.6891810695330302

Best val_loss So Far: 0.6891810695330302
Total elapsed time: 00h 03m 43s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
64                |128               |units
0.2               |0.2               |Dropout_rate
0.0001            |0.0001            |learning_rate

Epoch 1/100
86/86 [==============================] - 1s 7ms/step - loss: 1.3711 - accuracy: 0.2889 - val_loss: 1.3211 - val_accuracy: 0.3226
Epoch 2/100
86/86 [==============================] - 0s 3ms/step - loss: 1.2728 - accuracy: 0.4145 - val_loss: 1.2262 - val_accuracy: 0.5455
Epoch 3/100
86/86 [==============================] - 0s 3ms/step - loss: 1.1954 - accuracy: 0.5247 - val_loss: 1.1485 - val_accuracy: 0.6246
Epoch 4/100
86/86 [==============================] - 0s 3ms/step - loss: 1.1279 - accuracy: 0.5932 - val_loss: 1.0834 - val_accuracy: 0.6393
Epoch 5/100
86/86 [==============================] - 0s 3ms/step - loss: 1

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

In [ ]:
t_End=time.time()
t_elapsed=(t_End-t_Start)/60/60
print('Total execution : ',round(t_elapsed,2)) 
print(datetime.now())
print("=============================================================")

In [ ]:
# # Load the TensorBoard notebook extension
# %load_ext tensorboard
# # %tensorboard --logdir logs/fit 
# #%tensorboard --logdir logs/fit --host 0.0.0.0

# from tensorboard import notebook
# notebook.list() # View open TensorBoard instances
# # Control TensorBoard display. If no port is provided, 
# # the most recently launched TensorBoard is used
# notebook.display(port=6006, height=1000)

# Retain by best model to product model to production

In [77]:
print("Build the model with the optimal hyperparameters and train it on the data to find  N epochs")
best_model = tuner.hypermodel.build(best_hps)
# model.summary()
history = best_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, batch_size=BATCH_SIZE)

Build the model with the optimal hyperparameters and train it on the data to find  N epochs
Epoch 1/20
86/86 [==============================] - 2s 9ms/step - loss: 1.0880 - accuracy: 0.5225 - val_loss: 0.8661 - val_accuracy: 0.7009
Epoch 2/20
86/86 [==============================] - 0s 4ms/step - loss: 0.8315 - accuracy: 0.6840 - val_loss: 0.7561 - val_accuracy: 0.7067
Epoch 3/20
86/86 [==============================] - 0s 4ms/step - loss: 0.7556 - accuracy: 0.6990 - val_loss: 0.7245 - val_accuracy: 0.7097
Epoch 4/20
86/86 [==============================] - 0s 4ms/step - loss: 0.7325 - accuracy: 0.7016 - val_loss: 0.7130 - val_accuracy: 0.7126
Epoch 5/20
86/86 [==============================] - 0s 5ms/step - loss: 0.7087 - accuracy: 0.7162 - val_loss: 0.7098 - val_accuracy: 0.7185
Epoch 6/20
86/86 [==============================] - 0s 4ms/step - loss: 0.6991 - accuracy: 0.7129 - val_loss: 0.7051 - val_accuracy: 0.7243
Epoch 7/20
86/86 [==============================] - 0s 5ms/step - lo

In [78]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f'Best epoch: {best_epoch}')

Best epoch: 11


In [79]:
print("Fid Optimal Best Epoch  by Re-instantiating the hypermodel and train it")
model = tuner.hypermodel.build(best_hps)
# Retrain the model to get final
history_hypermodel=model.fit(train_ds, validation_data=val_ds, epochs=best_epoch)

val_loss, val_accuracy = model.evaluate(val_ds)
print(f"Eveluation on Val-Data at {best_epoch} epochs : loss={val_loss} and {main_metric}= {val_accuracy}")
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Evaluation on Test-Data at {best_epoch} epochs : loss={test_loss} and {main_metric}= {test_accuracy}")

Fid Optimal Best Epoch  by Re-instantiating the hypermodel and train it
Epoch 1/11
86/86 [==============================] - 2s 10ms/step - loss: 1.0336 - accuracy: 0.5610 - val_loss: 0.8403 - val_accuracy: 0.6979
Epoch 2/11
86/86 [==============================] - 0s 4ms/step - loss: 0.8115 - accuracy: 0.6877 - val_loss: 0.7490 - val_accuracy: 0.7038
Epoch 3/11
86/86 [==============================] - 0s 4ms/step - loss: 0.7534 - accuracy: 0.6990 - val_loss: 0.7288 - val_accuracy: 0.6979
Epoch 4/11
86/86 [==============================] - 0s 4ms/step - loss: 0.7237 - accuracy: 0.7096 - val_loss: 0.7177 - val_accuracy: 0.7067
Epoch 5/11
86/86 [==============================] - 0s 4ms/step - loss: 0.7073 - accuracy: 0.7115 - val_loss: 0.7153 - val_accuracy: 0.7126
Epoch 6/11
86/86 [==============================] - 0s 4ms/step - loss: 0.7015 - accuracy: 0.7082 - val_loss: 0.7091 - val_accuracy: 0.7097
Epoch 7/11
86/86 [==============================] - 0s 4ms/step - loss: 0.6857 - accura

In [140]:
def plot_metrics(history,metric):
    plt.figure(figsize=(15,8))
    plt.title(metric)
    plt.plot(history.history[metric], label='train')
    plt.plot(history.history[f'val_{metric}'], label='validation')
    plt.legend()
    plt.show() 

print("4#Explore Result model")
plot_metrics(history,main_metric)
plot_metrics(history,"loss")


4#Explore Result model


NameError: name 'history' is not defined

In [85]:
# model.save(model_tuned_dir)
# quit()

INFO:tensorflow:Assets written to: tuned_model\assets


INFO:tensorflow:Assets written to: tuned_model\assets


In [86]:
reloaded_model = tf.keras.models.load_model(model_tuned_dir)
#label_multi_severity
sample={"sla":"24x7 4Hrs Response Time",
        "product_type":"Server",
        "brand":"VMWare",                
        "service_type":"Incident",
        "incident_type":"General Incident",
        "open_to_close_hour":10,
        "response_to_resolved_hour":8.000000 \
       }

print(sample)
              
print("===============================================================================================================")    
print("convert pain data to serdor as input to predict")    
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
print(input_dict)

predictionList = reloaded_model.predict(input_dict)
print(predictionList)
prob = tf.nn.sigmoid(predictionList[0])
print(f"{(100 * prob)} %  as Severity") 

{'sla': '24x7 4Hrs Response Time', 'product_type': 'Server', 'brand': 'VMWare', 'service_type': 'Incident', 'incident_type': 'General Incident', 'open_to_close_hour': 10, 'response_to_resolved_hour': 8.0}
convert pain data to serdor as input to predict
{'sla': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'24x7 4Hrs Response Time'], dtype=object)>, 'product_type': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Server'], dtype=object)>, 'brand': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'VMWare'], dtype=object)>, 'service_type': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Incident'], dtype=object)>, 'incident_type': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'General Incident'], dtype=object)>, 'open_to_close_hour': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([10])>, 'response_to_resolved_hour': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([8.], dtype=float32)>}
1/1 [==============================] - 1s 529ms/step
[[0.00572414 0.1736307